In this notebook, I performed ETL and attempted to answer business questions related to the eCommerce data, specifically the transactions table. I connected the notebook to the database using psycopg2 and used my own queries (which can also be found in ../sql-queries/business-insights.sql) to derive insights. The queries range from simple to more complex, covering various aspects of the business.

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError

import psycopg2
from psycopg2 import OperationalError

import pandas as pd

In [2]:
def create_conn():
    conn = None
    try:
        conn = psycopg2.connect(
            database="ecommerce",
            user="postgres",
            password="superadmin",
            host="localhost",
            port="5432",
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return conn

engine = create_conn()

Connection to PostgreSQL DB successful


In [3]:
def query_data_from_db(engine, query):
    """
    Perform data transformation from a database using a given SQL query
    Args:
        connection: A connection object
        query (str): SQL query
    Returns:
        df: A dataframe containing the transformed data
    """
    df = None
    try:
        df = pd.read_sql_query(query, con=engine)
        print("Data fetched successfully.")
    except Exception as e:
        print(f"Error occurred during data fetching: {e}")
    return df

In [5]:
#  Query to find the top 10 products that generate the highest revenue. 
#  This helps to understand which products contribute most to the company's income.

query = """
    select  
        hits_product_productsku, 
        hits_product_v2productname, 
        SUM(hits_product_productrevenue) as total_revenue
    from 
        all_transactions
    where 
        hits_product_productsku is not null
    group by 
        hits_product_productsku, hits_product_v2productname
    order by
        total_revenue desc
    limit 10;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,hits_product_productsku,hits_product_v2productname,total_revenue
0,GGOENEBQ078999,Nest® Cam Outdoor Security Camera - USA,3.097986e+11
1,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,2.562240e+11
2,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,2.175993e+11
3,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,9.551879e+10
4,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,9.011329e+10
5,GGOENEBJ081899,Nest® Learning Thermostat 3rd Gen - CA - Stain...,3.520532e+10
6,GGOENEBQ084699,Nest® Learning Thermostat 3rd Gen-USA - White,1.675046e+10
7,GGOEGOLC014299,Google Metallic Notebook Set,1.603642e+10
8,GGOENEBB081499,Nest® Cam Indoor Security Camera - CA,1.472483e+10
9,GGOEGDHQ015399,26 oz Double Wall Insulated Bottle,1.369203e+10


In [7]:
# -- Query to find the top 10 cities that generate the highest number of transactions. 
# -- This gives an overview of the geographical distribution of the company's sales.

query = """
    select 
        geonetwork_city, 
        COUNT(*) as number_of_transactions
    from
        all_transactions
    where 
        geonetwork_city is not null and geonetwork_city <> 'not available in demo dataset'
    group by 
        geonetwork_city
    order by 
        number_of_transactions desc 
    limit 10;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,geonetwork_city,number_of_transactions
0,Mountain View,2055
1,New York,1909
2,San Francisco,1218
3,Sunnyvale,855
4,Chicago,692
5,San Jose,498
6,Los Angeles,465
7,Seattle,431
8,Toronto,429
9,Palo Alto,425


In [8]:
# -- Query to find the top 10 countries that generate the highest number of transactions. 
# -- This could inform international marketing strategies.

query = """
	select 
		geonetwork_country, 
		COUNT(*) as number_of_transactions
	from
		all_transactions
	where 
		geonetwork_country is not null and geonetwork_country <> 'not available in demo dataset'
	group by 
		geonetwork_country
	order by 
		number_of_transactions desc 
	limit 10;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,geonetwork_country,number_of_transactions
0,United States,17145
1,Canada,1038
2,Venezuela,499
3,Mexico,110
4,Taiwan,93
5,Japan,86
6,Australia,77
7,Singapore,65
8,Indonesia,55
9,Brazil,47


In [9]:
# -- Query to find the average time a user spends on the site. 
# -- This is a useful indicator of user engagement.

query = """
	select 
		ROUND(AVG(totals_timeonsite), 2) as average_time_spend_seconds,
		ROUND(AVG(totals_timeonsite) / 60, 2) as average_time_spend_minutes
	from
		all_transactions;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,average_time_spend_seconds,average_time_spend_minutes
0,1369.19,22.82


In [10]:
# -- Query to find the top 10 products with the highest revenue per transaction. 
# -- This could help identify high-value products.

query = """
	select
		hits_product_productsku,
		hits_product_v2productname,
		SUM(hits_product_productrevenue) / COUNT(hits_transaction_transactionid) as revenue_per_transaction
	from
		all_transactions 
	where 
		hits_product_productsku is not null
	group by
		hits_product_productsku,
		hits_product_v2productname
	order by 
		revenue_per_transaction desc 
	limit 10;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,hits_product_productsku,hits_product_v2productname,revenue_per_transaction
0,GGOEGAEL031114,Google Men's Heavyweight Long Sleeve Hero Tee ...,1.480230e+09
1,GGOEGAEL031117,Google Men's Heavyweight Long Sleeve Hero Tee ...,1.120320e+09
2,GGOEGAUB058315,Google Women's Performance Polo Grey/Black,1.071143e+09
3,GGOEGAUB058313,Google Women's Performance Polo Grey/Black,8.191825e+08
4,GGOEGAEL031113,Google Men's Heavyweight Long Sleeve Hero Tee ...,7.582321e+08
5,GGOEGAUB058314,Google Women's Performance Polo Grey/Black,7.390413e+08
6,GGOEGAUB058316,Google Women's Performance Polo Grey/Black,6.932025e+08
7,GGOEGAEL031115,Google Men's Heavyweight Long Sleeve Hero Tee ...,6.138767e+08
8,GGOEGAEL031116,Google Men's Heavyweight Long Sleeve Hero Tee ...,5.765526e+08
9,GGOEGEVR014999,UFO Bluetooth Water Resistant Speaker,4.911725e+08


In [11]:
# -- Query to find the top 10 products with the most transactions. 
# -- This could help identify popular products.

query = """
	select
		hits_product_productsku,
		hits_product_v2productname,
		COUNT(hits_transaction_transactionid) as number_of_trasaction
	from
		all_transactions 
	inner join
		product_list pl 
	on
		all_transactions.hits_product_productsku = pl.productsku
	group by 
		hits_product_productsku,
		hits_product_v2productname
	order by 
		number_of_trasaction desc 
	limit 10;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,hits_product_productsku,hits_product_v2productname,number_of_trasaction
0,GGOENEBQ078999,Nest® Cam Outdoor Security Camera - USA,1443
1,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,1149
2,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,993
3,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,623
4,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,455
5,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,275
6,GGOEGDHC018299,Google 22 oz Water Bottle,188
7,GGOEGHGC019799,Google Sunglasses,168
8,GGOEGDHQ015399,26 oz Double Wall Insulated Bottle,161
9,GGOEGODR017799,Recycled Mouse Pad,154


In [12]:
# -- Query to find which cities have the highest average transaction revenue. 
# -- This could help target marketing efforts in regions with high spending power.

query = """
	select 
		geonetwork_city,
		ROUND(AVG(totals_totaltransactionrevenue)) as avg_transaction_revenue
	from 
		all_transactions 
	group by
		geonetwork_city
	having
		COUNT(*) > 100
	order by 
		avg_transaction_revenue desc 
	limit 10;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,geonetwork_city,avg_transaction_revenue
0,Salem,3.277344e+09
1,Cambridge,6.983861e+08
2,Toronto,5.093308e+08
3,not available in demo dataset,4.736208e+08
4,Maracaibo,4.304613e+08
5,Chicago,3.907384e+08
6,Palo Alto,3.481369e+08
7,Los Angeles,3.468151e+08
8,San Bruno,3.090448e+08
9,Atlanta,2.952540e+08


In [13]:
# -- More complex query that uses window functions to find the top 3 products (by revenue) for each city. 
# -- This can provide insights into regional product preferences.

query = """
	select 
		city_product_rank.geonetwork_city,
		city_product_rank.hits_product_productsku,
		city_product_rank.total_revenue,
		city_product_rank.product_rank
	from
		(
			select
				geonetwork_city,
				hits_product_productsku,
				SUM(hits_product_productrevenue) as total_revenue,
				RANK() over (
					partition by geonetwork_city
					order by SUM(hits_product_productrevenue) desc
				) as product_rank
			from 
				all_transactions
			where 
				geonetwork_city is not null and geonetwork_city not in ('not available in demo dataset', '(not set)')
			group by
				geonetwork_city,
				hits_product_productsku
		) as city_product_rank
	where 
		city_product_rank.product_rank <= 3;
"""

result_df = query_data_from_db(engine, query)

result_df

Data fetched successfully.


C:\Users\PF2L6BL6\AppData\Local\Temp\ipykernel_1744\794050568.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=engine)


,geonetwork_city,hits_product_productsku,total_revenue,product_rank
0,Ahmedabad,GGOEYOLR018699,7865000.0,1
1,Ahmedabad,GGOEGOBG023599,3865000.0,2
2,Ahmedabad,GGOEGFKQ020399,3865000.0,2
3,Ahmedabad,GGOEAFKQ020599,3865000.0,2
4,Amsterdam,GGOENEBJ079499,302000000.0,1
...,...,...,...,...
328,Yokohama,GGOEGAXC065228,17990000.0,1
329,Yokohama,GGOEGAXJ065528,12890000.0,2
330,Zurich,GGOENEBJ079499,304000000.0,1
331,Zurich,GGOEGADJ057118,91990000.0,2


In [14]:
engine.close()